### Goal of this notebook: To replicate the Map produced by the Climate Cabinet (Overlay the Justice40 comunites geographics onto the Utility Territories)
#### 1. Load both the CSV datasets of Utility Territories and Justice40 tract level data.
#### 2. Check if they can be merged using the native pandas, by identifying common keys if any
#### 3. If that cannot be done, proceed with the shape files from the web.
#### 4. Overlay the Justice40 data from the shapefile onto the Utility Territories
#### 5. Save the Overlay as a GeoJSON file and make a plot of the dataframe

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [ ]:
wd = 'C:\\Users\\saikr\\Desktop\\CDF\\Climate Cabinet\\'

In [ ]:
util_df = pd.read_csv(wd+ 'utility_territories.csv')
util_df.shape

In [ ]:
util_df.info()
#Shows it has no nulls, but needs to be checked if they are encoded as something else

In [ ]:
util_df.head(3)

#### The utility territories data do not have any tract information on it, but just the City, State, etc as the geographical data. All the gro columns have a lot of nulls in them coded as ``"NOT AVAILABLE"``. ID column - Is it the tract ID or a different ID?

In [ ]:
util_df[util_df['CITY']=='NOT AVAILABLE'].shape

In [ ]:
justice_df = pd.read_csv(wd+'justice40.csv', low_memory=False)

In [ ]:
justice_df.shape

In [ ]:
justice_df.head(2)

#### This file has the tract and county details but still has no matching keys with the Utility Territories data. The total tract level population is the main feature that should be labelled in the plot

In [ ]:
## Read the shape files since both csv's donot have any matching keys that can link the datasets together

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
%%time
justice40 = gpd.read_file(wd +'justice40 shapefile\\usa\\usa.shp')
utility_territories = gpd.read_file(wd + 'utility shape file\\Electric_Retail_Service_Territories.shp')

#### Starting with cleaning the Justice40 Data

In [ ]:
%%time
## Make sure both datasets have same coordinate reference system
justice40 = justice40.to_crs(utility_territories.crs)

#### Select only the necessary columns from the Justice40 data (Refer to the columns csv)

In [ ]:
justice40.head(2)

In [ ]:
cols_to_keep = ['GEOID10','SF', 'CF','TPF','UF_PFS','SN_C','SN_T','HRS_ET','LHE','FPL200S','UI_EXP','THRHLD','geometry']
justice40 = justice40[cols_to_keep]
justice40.shape

In [ ]:
justice40.columns = ['GeoID','State','County','Tot_Pop','Unemp_Per','Disadv?','Disad_Tri?','Underinves','Low_HS_Edu','Low_Income','UI_EXP','THRHLD','geometry']
justice40.head(2)

In [ ]:
%%time
#save the cleaned justice40 dataset
justice_clean_path = wd+'justice40_clean.shp'

# Export the GeoDataFrame as a shapefile
justice40.to_file(justice_clean_path, driver='ESRI Shapefile')

In [ ]:
%%time
justice_clean = gpd.read_file(justice_clean_path)
justice_clean.head(2)

In [ ]:
#check for duplicates
justice_clean.duplicated('GeoID').sum()

#### EDA for the truncated version of Justice40 data

In [ ]:
plt.figure(figsize = (8,5))
justice_nulls = pd.DataFrame(justice_clean.isnull().sum()/justice_clean.shape[0] * 100, columns = ['Nulls%'])
sns.barplot(x = justice_nulls.index, y = justice_nulls['Nulls%'])
plt.xticks(rotation = 90, fontsize = 8)
plt.title('Nulls Percentage in the Justice40 Data')
plt.show()

In [ ]:
justice_nulls.sort_values(by = 'Nulls%', ascending = False)
## The nulls are in the Underinvested and Disadvantaged_Tribal columns, which should not used as a filter or condition later on. Can be dropped if needed

In [ ]:
plt.figure(figsize=(10,5))
sns.heatmap(justice_clean.isnull(), yticklabels=False)
plt.title('Null Values Heat Map for the Justice40 Data')
plt.show()

In [ ]:
uni_tot_df_just = pd.DataFrame(columns = ['Column', 'Total', 'Unique', 'All Unique' ,'Missing'])
j = 0
for i in justice_clean.columns:
    tot = justice_clean[justice_clean[i].notnull()].shape[0]
    uni = justice_clean[i].nunique()
    mis = (1 - (tot / justice_clean.shape[0]) ) * 100
    au = tot == uni
    uni_tot_df_just.loc[j] = [i, tot, uni,au, mis]
    j = j+1
    

In [ ]:
### Identify the Total Count, Uniques Count, Missing Percentage in the Justice40 Data
uni_tot_df_just = uni_tot_df_just.transpose()
uni_tot_df_just.columns = uni_tot_df_just.iloc[0]
uni_tot_df_just = uni_tot_df_just.iloc[1:]
uni_tot_df_just

In [ ]:
justice_clean['Disadv?'].value_counts()

In [ ]:
sns.countplot(x =justice_clean['Disadv?'])
plt.title('Disadvantaged Communities in the Justice40 Data')
plt.xlabel('Disadvantaged?')

In [ ]:
# Group by "State" and calculate the sum of "Disadv?" column
state_disadv_sum = justice_clean.groupby('State')['Disadv?'].sum()

plot_data = pd.DataFrame({'State': state_disadv_sum.index, 'Disadv_Sum': state_disadv_sum.values})

plt.figure(figsize=(10, 6))
sns.barplot(y='Disadv_Sum', x='State', data=plot_data)
plt.title('State-Level Disadvantage Communities')
plt.ylabel('Population')
plt.xlabel('State')
plt.xticks(rotation=90)
plt.show()

#### Moving onto Service Utility Territories data

In [ ]:
utility_territories.columns

In [ ]:
util_cols_to_remove = ['SOURCE','SOURCEDATE','VAL_METHOD','VAL_DATE']
utility_territories.drop(util_cols_to_remove, axis = 1, inplace = True)
utility_territories.shape

In [ ]:
%%time
#save the cleaned dataset
util_clean_path = wd+'util_clean.shp'

# Export the GeoDataFrame as a shapefile
utility_territories.to_file(util_clean_path, driver='ESRI Shapefile')

In [ ]:
utility_territories.head(2)

#### EDA for the Utility Territories data

In [ ]:
plt.figure(figsize = (8,5))
util_nulls = pd.DataFrame(utility_territories.isnull().sum()/utility_territories.shape[0] * 100, columns = ['Nulls%'])
sns.barplot(x = util_nulls.index, y = util_nulls['Nulls%'])
plt.xticks(rotation = 90, fontsize = 8)
plt.title('Nulls Percentage in the Utility Territories Data')
plt.show()

In [ ]:
uni_tot_df_util = pd.DataFrame(columns = ['Column', 'Total', 'Unique', 'All Unique' ,'Missing'])
j = 0
for i in utility_territories.columns:
    tot = utility_territories[utility_territories[i].notnull()].shape[0]
    uni = utility_territories[i].nunique()
    mis = (1 - (tot / utility_territories.shape[0]) ) * 100
    au = tot == uni
    uni_tot_df_util.loc[j] = [i, tot, uni,au, mis]
    j = j+1
    

In [ ]:
### Identify the Total Count, Uniques Count, Missing Percentage in the Utility Territories Data
uni_tot_df_util = uni_tot_df_util.transpose()
uni_tot_df_util.columns = uni_tot_df_util.iloc[0]
uni_tot_df_util = uni_tot_df_util.iloc[1:]
uni_tot_df_util

In [ ]:
# Group by "STATE" and calculate the number of Utilities per state
state_util_count = utility_territories.groupby('STATE')['OBJECTID'].count()

# Create a new DataFrame with state names and sums
plot_data = pd.DataFrame({'State': state_util_count.index, 'Count': state_util_count.values})

# Plot the heatmap
plt.figure(figsize=(10, 6))
sns.barplot(y='Count', x='State', data=plot_data)
plt.title('State-Level Utility Providers')
plt.ylabel('Count')
plt.xlabel('State')
plt.xticks(rotation=90)
plt.show()

In [ ]:
util_clean = gpd.read_file(util_clean_path)

In [ ]:
justice_clean.head(2)

In [ ]:
util_clean.head(2)

In [ ]:
util_clean.plot(figsize=(10, 10), color='lightgray', edgecolor='black')
justice_clean.plot(ax=plt.gca(), color='blue', alpha=0.5)
plt.title('Utility Territories with Justice40 Communities Overlay')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend(['Utility Territories', 'Justice40 Communities'])
plt.show()


In [ ]:
# import folium
# import geopandas as gpd

# # Reproject utility_territories to a projected CRS
# util_clean = util_clean.to_crs('EPSG:3857')

# # Filter out rows with invalid geometries in justice_clean
# valid_justice = justice_clean[justice_clean.geometry.is_valid]

# # Create a folium map centered on the mean coordinates of the utility territories
# map_center = util_clean.geometry.centroid.unary_union.centroid.coords[0]
# m = folium.Map(location=[map_center[1], map_center[0]], zoom_start=10)

# # Add the utility territories as polygons to the map
# folium.GeoJson(util_clean, name='Utility Territories', style_function=lambda x: {'fillColor': 'lightgray', 'color': 'black'}).add_to(m)

# # Add the justice40 communities as markers to the map
# for idx, row in valid_justice.iterrows():
#     folium.CircleMarker(
#         location=[row.geometry.centroid.y, row.geometry.centroid.x],
#         radius=5,
#         color='blue',
#         fill=True,
#         fill_color='blue',
#         fill_opacity=0.5,
#         tooltip=f"Community: {row['GeoID']}"
#     ).add_to(m)

# # Add layer control to toggle between utility territories and justice40 communities
# folium.LayerControl().add_to(m)

# # Display the map
# m
